# Redes Neuronales Recurrentes (RNN)

## 1. Vectorización de texto

### 1.1. One-hot encoding

In [3]:
from numpy import array
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

doc = "Me gusta el Deep Learning"

doc = doc.lower()
doc = doc.split()
values = array(doc)
print(values)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

['me' 'gusta' 'el' 'deep' 'learning']
[4 2 1 0 3]
[[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


c:\Users\pepel\.conda\envs\PySpace310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### 1.2. Embedding layer de Keras

In [4]:
from tensorflow.keras import layers

layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)

## 2. Programando una RNN: Generación de texto

### 2.1. Descarga y preprocesado de los datos

In [5]:
import os

path_txt = os.path.join("DeepLearning-Introduccion-practica-con-Keras-PRIMERA-PARTE.txt")

text = open(path_txt, 'rb').read().decode(encoding='utf-8')
print("Longitud del texto: {} caráceters".format(len(text)))

vocab = sorted(set(text))
print("El texto está compuesto de estos {} caráceteres:".format(len(vocab)))
print(f"\n{vocab}")

Longitud del texto: 203251 caráceters
El texto está compuesto de estos 92 caráceteres:

['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


In [6]:
import numpy as np

char2dix = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [7]:
for char, _ in zip(char2dix, range(len(vocab))):
    print(" {:4s}: {:3d},".format(repr(char), char2dix[char]))

 '\n':   0,
 '\r':   1,
 ' ' :   2,
 '!' :   3,
 '"' :   4,
 '#' :   5,
 '%' :   6,
 "'" :   7,
 '(' :   8,
 ')' :   9,
 '*' :  10,
 '+' :  11,
 ',' :  12,
 '-' :  13,
 '.' :  14,
 '/' :  15,
 '0' :  16,
 '1' :  17,
 '2' :  18,
 '3' :  19,
 '4' :  20,
 '5' :  21,
 '6' :  22,
 '7' :  23,
 '8' :  24,
 '9' :  25,
 ':' :  26,
 ';' :  27,
 '<' :  28,
 '=' :  29,
 '>' :  30,
 '?' :  31,
 '@' :  32,
 'A' :  33,
 'B' :  34,
 'C' :  35,
 'D' :  36,
 'E' :  37,
 'F' :  38,
 'G' :  39,
 'H' :  40,
 'I' :  41,
 'J' :  42,
 'K' :  43,
 'L' :  44,
 'M' :  45,
 'N' :  46,
 'O' :  47,
 'P' :  48,
 'Q' :  49,
 'R' :  50,
 'S' :  51,
 'T' :  52,
 'U' :  53,
 'V' :  54,
 'W' :  55,
 'X' :  56,
 'Y' :  57,
 '[' :  58,
 ']' :  59,
 '_' :  60,
 'a' :  61,
 'b' :  62,
 'c' :  63,
 'd' :  64,
 'e' :  65,
 'f' :  66,
 'g' :  67,
 'h' :  68,
 'i' :  69,
 'j' :  70,
 'k' :  71,
 'l' :  72,
 'm' :  73,
 'n' :  74,
 'o' :  75,
 'p' :  76,
 'q' :  77,
 'r' :  78,
 's' :  79,
 't' :  80,
 'u' :  81,
 'v' :  82,
 'w'

In [8]:
text_as_int = np.array([char2dix[c] for c in text])

In [9]:
print("texto: {}".format(repr(text[:50])))
print("{}".format(repr(text_as_int[:50])))

texto: 'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun'
array([48, 78, 75, 72, 75, 67, 75,  1,  0, 37, 74,  2, 17, 25, 21, 19, 12,
        2, 41, 79, 61, 61, 63,  2, 33, 79, 69, 73, 75, 82,  2, 76, 81, 62,
       72, 69, 63, 75,  2, 51, 65, 67, 81, 74, 64, 61,  2, 38, 81, 74])


### 2.2. Preparación de los datos para ser usados por la RNN

In [10]:
import tensorflow as tf

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
for item in sequences.take(10):
    print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '
'un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo,'
' que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de'
' nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de '
'aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Proces'
'sor Units), que solo cuestan alrededor de 100 euros, que esta

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in dataset.take(1):
    print("Input data: ", repr(''.join(idx2char[input_example.numpy()])))
    print("Target data: ", repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion'
Target data:  'rologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '


In [14]:
print(dataset)

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>


In [15]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>


### 2.3. Construcción del modelo RNN

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()
    model.add(Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))
    model.add(LSTM(
        rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer='glorot_uniform'
    ))
    model.add(Dense(vocab_size))
    return model

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           23552     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 92)            94300     
                                                                 
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
    print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 92) # (batch_size, sequence_length, vocab_size)


In [20]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],
    num_samples=1)

sampled_indices_characters = tf.squeeze(
    sampled_indices,
    axis=-1
).numpy()

In [21]:
print(sampled_indices_characters)

[21 50  7 66  0  3 55 64 14 71 79 17 61 78 35 14 61 16 10  2 58 71 55 62
 74 40  7 43 23 10 29 55 41 12 60 57 23 74  5 90 26 71 61 26 24 53 88 81
 16 34 76 29 11  0 43 44 15 50 91 47 60 38 72 25 68 14 55 47  4 21 64 85
 70  6 38 28  7 69 61 91 90 31 91 28 26 61 13 23 47 53 56 60  8 65 90 49
 12 56 21 54]


### 2.4. Entrenamiento del modelo RNN

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels,
        logits,
        from_logits=True
    )

In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "cpkt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [25]:
EPOCHS = 50
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

Epoch 1/50
31/31 [==============================] - 2s 29ms/step - loss: 3.2831
Epoch 2/50
31/31 [==============================] - 1s 28ms/step - loss: 2.8602
Epoch 3/50
31/31 [==============================] - 1s 28ms/step - loss: 2.5385
Epoch 4/50
31/31 [==============================] - 1s 28ms/step - loss: 2.3278
Epoch 5/50
31/31 [==============================] - 1s 28ms/step - loss: 2.1978
Epoch 6/50
31/31 [==============================] - 1s 28ms/step - loss: 2.1068
Epoch 7/50
31/31 [==============================] - 1s 28ms/step - loss: 2.0137
Epoch 8/50
31/31 [==============================] - 1s 28ms/step - loss: 1.9147
Epoch 9/50
31/31 [==============================] - 1s 28ms/step - loss: 1.8174
Epoch 10/50
31/31 [==============================] - 1s 28ms/step - loss: 1.7198
Epoch 11/50
31/31 [==============================] - 1s 28ms/step - loss: 1.6319
Epoch 12/50
31/31 [==============================] - 1s 28ms/step - loss: 1.5492
Epoch 13/50
31/31 [==================

### 2.5. Generación de texto usando el modelo RNN

In [26]:
model = build_model(
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size=1
)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
def generate_text(model, start_string):
    num_generate = 500
    input_eval = [char2dix[s] for s in start_string]

    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.5

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)

        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions,
            num_samples=1
        )[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
    
    return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string="Domingo "))

Domingo que contiene el trazo de neuronas de la capa oculta con las GPU en un nuevo elemento X1 y por un solo el minimo global de la inteligencia artificial en Keras los actualmente este es la que obtiene en una nueva fuente de creacion de validacion introducida variables de entrada (features) utilizando una medida de la inteligencia artificial y la funcion de activacion softmax
Vamos a resolver el problema de optimizacion sobre el tema del conjunto de datos de tener el proceso de aprendizaje de una re


In [34]:
print(generate_text(model, start_string="Activacion "))

Activacion permite conectar tambien GPU y ed netro los sesgos que se consigue con el tamano de la segunda parte del libro.
Precarga el ?pooling tiende se muestra los conocidos en cada epochs se permitir en los siguientes capitulos que tenemos un modelo que puede ser de ayuda al lector para ver como se codigo que presentaremos en el tema y mas complejo. 
Pero en realidad el calculo de parametros para los casos en cada iteracion, una vez aprendizaje con la accuracy de los datos de entrenamiento poco estan 


In [35]:
print(generate_text(model, start_string="Redes "))

Redes nuestro modelo de referencia se ejecuta como se comporta con el que se esta es usar un grismo mis magiente puede definir en la imagen del docker.
Para este articulo de referencia del capitulo anterior y permite reducir valores diendo los puntos anarieras en el ano 201251, corresponde a la capa de entrada (input layer), una o mas capas convolucionales que pueden ser aprenden este ejemplo simple, intentando como el supercomputador Marenostrum que se ajustara nos encontramos ante un nuevo valor de


In [39]:
print(generate_text(model, start_string="a"))

an la capa de neuronas. Mis a partir de la humanidad es comparable con la loss de cada capa oculta que contribuyen directamente a continuacion. 
En el proceso de entrenamiento contendra una capa convolucional su primer ejemplo de clasificacion que sigue al modelo y cada uno de los que ya hay disponible en dia es que en general, los de validacion con la imagen de entrada y de la prediccion de la red neuronal convolucional completa con la inteligencia artificial en general, y en cada capitulo. 
E
